# Walkthrough for using the Fast Stylometry model for Burrows' Delta

You can run this notebook in Google Colab: <a href="https://colab.research.google.com/github/fastdatascience/faststylometry/blob/main/Burrows%20Delta%20Walkthrough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

By [Thomas Wood](https://freelancedatascientist.net), [Fast Data Science](https://fastdatascience.com)

Burrows' Delta is an algorithm for comparing the similarity of the writing styles of documents, known as [forensic stylometry](https://fastdatascience.com/how-you-can-identify-the-author-of-a-document/).

* [A useful explanation of the maths and thinking behind Burrows' Delta and how it works](https://programminghistorian.org/en/lessons/introduction-to-stylometry-with-python#third-stylometric-test-john-burrows-delta-method-advanced)

Demonstration of Burrows' Delta on a small corpus downloaded from Project Gutenberg.

We will test the Burrows' Delta code on two "unknown" texts: Sense and Sensibility by Jane Austen, and Villette by Charlotte Bronte. Both authors are in our training corpus.

This notebook demonstrates how to use the library to calculate the Burrows' Delta value of six candidate authors, and also to calculate the probability that each one is the author of the mystery text.


You can also read the [tutorial](https://fastdatascience.com/fast-stylometry-python-library/) or try out the [live demo of the tool](https://fastdatascience.com/forensic-stylometry-linguistics-authorship-analysis-demo/).

## First install the library

In [1]:
!pip install faststylometry

In [2]:
from faststylometry import Corpus

from faststylometry import download_examples
from faststylometry import load_corpus_from_folder
from faststylometry import tokenise_remove_pronouns_en
from faststylometry import calculate_burrows_delta
from faststylometry import predict_proba, calibrate, get_calibration_curve


KeyboardInterrupt



Download the example corpus (Project Gutenberg texts) from the internet

In [ ]:
download_examples()

Load the data into Fast Stylometry library.

In [ ]:
train_corpus = load_corpus_from_folder("data/train")

train_corpus.tokenise(tokenise_remove_pronouns_en)

# Load two books by "unknown" authors to test the algorithm

In [ ]:
# Load Sense and Sensibility, written by Jane Austen (marked as "janedoe")
# and Villette, written by Charlotte Bronte (marked as "currerbell", Bronte's real pseudonym)

test_corpus = load_corpus_from_folder("data/test", pattern=None)
# You can set pattern to a string value to just load a subset of the corpus.

test_corpus.tokenise(tokenise_remove_pronouns_en)

# Calculate Burrows' Delta for both candidate authors

In [ ]:
calculate_burrows_delta(train_corpus, test_corpus, vocab_size = 50)

# Calibrate the model and calculate the probability of each candidate in the training set being the author

In [ ]:
calibrate(train_corpus)

In [ ]:
predict_proba(train_corpus, test_corpus)

# Plot the calibration curve

We have used Scikit Learn's Logistic Regression to calculate the calibration curve of the model. You could also use your own calibration curve.

In [ ]:
import numpy as np
x_values =  np.arange(0, 3, 0.1)

import matplotlib.pyplot as plt

In [ ]:
plt.plot(x_values, train_corpus.probability_model.predict_proba(np.reshape(x_values, (-1, 1)))[:,1])
plt.xlabel("Burrows delta")
plt.ylabel("Probability of same author")
plt.title("Calibration curve of the Burrows Delta probability model\nUsing Logistic Regression with correction for class imbalance")
;

# Plot the ROC curve

The ROC curve and the AUC metric are useful ways of measuring the performance of a classifier.

In our case the Burrows' Delta method, when used as a two-class text classifier (different author vs. same author), has an incredibly easy task, because it has learnt from entire books.

In [ ]:
ground_truths, deltas = get_calibration_curve(train_corpus)

In [ ]:
probabilities = train_corpus.probability_model.predict_proba(np.reshape(deltas, (-1, 1)))[:,1]

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
fpr, tpr, thresholds = roc_curve(ground_truths, probabilities)

In [ ]:
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure()
plt.plot(fpr, tpr, color='darkorange',
         label='ROC curve (area = %0.4f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating curve of the Burrows\' Delta classifier\noperating on entire books')
plt.legend(loc="lower right")
plt.show()

# Segment the corpus and display the various books in a scatter graph using PCA

Scatter graph shows books' stylistic similarities by placing similar books or segments close together in 2D space.

In [ ]:
from sklearn.decomposition import PCA
import re
import pandas as pd

In [ ]:
# Reload the training corpus as the "test corpus", re-tokenise it, and segment it this time
test_corpus = load_corpus_from_folder("data/train")
test_corpus.tokenise(tokenise_remove_pronouns_en)

In [ ]:
split_test_corpus = test_corpus.split(80000)

In [ ]:
df_delta = calculate_burrows_delta(train_corpus, split_test_corpus)

In [ ]:
z_scores = split_test_corpus.author_z_scores

In [ ]:
pca_model = PCA(n_components=2)

In [ ]:
pca_matrix = pca_model.fit_transform(z_scores.T)

In [ ]:
authors = split_test_corpus.authors

In [ ]:
df_pca_by_author = pd.DataFrame(pca_matrix)
df_pca_by_author["author"] = authors

In [ ]:
plt.figure(figsize=(15,15)) 

for author, pca_coordinates in df_pca_by_author.groupby("author"):
    plt.scatter(*zip(*pca_coordinates.drop("author", axis=1).to_numpy()), label=author)
for i in range(len(pca_matrix)):
    plt.text(pca_matrix[i][0], pca_matrix[i][1],"  " + authors[i], alpha=0.5)

plt.legend()

plt.title("Representation using PCA of works in training corpus")